In [28]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature # Sets the feature attribute of the node
        self.threshold = threshold # Represents the threshold falue for the feature
        self.left = left # Refers to the left child node of the current node
        self.right = right # Refers to the right child node of the current node
        self.value = value # Intended to store the prediction value for a leaf node
    
    def is_leaf_node(self):
        return self.value is not None # Returns a boolean value which checks whether the value attribute of the node is not None
    

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None): # Definess the constructor of the decision tree class
        self.min_samples_split = min_samples_split # This controls the minimum number of samples required to split an internal node
        self.max_depth = max_depth # Is the length of the longest path from the root to a leaf
        self.n_features = n_features # used to specify the number of features to conisder for looking for the best split at each node
        self.root = None # Intended to store the root node of the decision tree once it is built

    def fit(self, x, y):
        self.n_features = x.shape[1 ] if not self.n_features else min(x.shape[1], self.n_features) # Ensures n_features is set correctly
        self.root = self._grow_tree(x, y) # Initializes the root attribute by calling the internal method _grow_tree(x, y)

    def _grow_tree(self, x, y, depth=0):
        n_samples, n_feats = x.shape # Extracts the number of samples and features from x data matrix
        n_labels = len(np.unique(y)) # Calculates the number of unique classes or labels in array y

        if(depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split): # Checks the current depth, if there is only one label in array y, and the number of samples
            leaf_value = self._most_common_label(y) 
            return Node(value=leaf_value)
        
        feat_idx = np.random.choice(n_feats, self.n_features, replace=False) # Each integer represents an index that is randomly selected and that number can't be used again
        
        best_feature, best_thresh = self._best_split(x, y, feat_idx) # Determines the best feature and threshold for splitting the dataset. 

        left_idxs, right_idxs = self._split(x[:, best_feature], best_thresh) # 2D structure that slices the array to get the values for the best features across all samples
        left = self._grow_tree(x[left_idxs, :], y[left_idxs], depth+1) # Part of the recursive tree building process that creates the left tree branch 
        right = self._grow_tree(x[right_idxs, :], y[right_idxs], depth+1) # Part of the recursive tree building process that created the right tree branch
        return Node(best_feature, best_thresh, left, right) 

    def _most_common_label(self, y):
        counter = Counter(y) # Counts hashable objects 
        value = counter.most_common(1)[0][0] # Indexes into a tuple to get the first value. Used to determine the majority class at a leaf node
        return value

    def _best_split(self, x, y, feat_idxs):
        best_gain = -1 # Ensures any actual gain whether positive or negative will be larger
        split_index, split_threshold = None, None 

        for feat_idx in feat_idxs: # Iterates over each feature index
            x_column = x[:, feat_idx] # Extracts the entire column from the dataset x
            thresholds = np.unique(x_column) # Finds all unique values in the selected feature column 

            for thr in thresholds: # Iterates over each unique threshold value in the current feature column  
                gain = self._information_gain( y, x_column, thr) # calculates the information gain

                if gain > best_gain: # Checks if the information gain is greater than the best gain found so far
                    best_gain = gain 
                    split_idx = feat_idx
                    split_threshold = thr
                    
        return split_idx, split_threshold
    
    def _information_gain(self, y, x_column, threshold):
        parent_entropy = self._entropy(y)

        left_idxs, right_idxs = self._split(x_column, threshold) # Divides the indices of the dataset into two groups 

        if len(left_idxs) == 0 or len(right_idxs) == 0: # Checks if right or left indicies are empty 
               return 0 
        
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs) # the number of samples that fall into the left and right group after the split
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs]) # The target variable in the left and right indicies are calculated here
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r # The weighted average of the two indicies are calculated here

        information_gain = parent_entropy - child_entropy 
        return information_gain

        
    def _split(self, x_column, split_threshold):
        # The .flatten() is used to convert the 2D array into a 1D array
        left_idxs = np.argwhere(x_column <= split_threshold).flatten() # Finds the indicies of all samples in x_samples. Results in a 2D array where each row is a list containing the index of a satisfying element. 
        right_idxs = np.argwhere(x_column > split_threshold).flatten() # Finds the indicies of all samples in x_samples. Results in a 2D array where each row is a list containing the index of a satisfying element. 
        return left_idxs, right_idxs


    def _entropy(self, y):
        y_int = y.astype(int) # Coverts y into an array of integers
        hist = np.bincount(y_int) # Computes a histogram of non negative integers
        ps = hist / len(y_int) # Calculates the probability of each unique label
        return -np.sum([p * np.log2(p) for p in ps if p>0]) # Computes the entropy using the Shannon entropy formula
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X]) # Iterates over each sample x in the array x
    
    def _traverse_tree(self, x, node): 
        if node.is_leaf_node(): # Checks if the current node is a leaf node
            return node.value
        
        if x[node.feature] <= node.threshold: 
            return self._traverse_tree(x, node.left) 
        return self._traverse_tree(x, node.right)

In [35]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


data = datasets.load_breast_cancer() # Calls in dataset from sklearn
x, y = data.data, data.target # Unpacks the data into the variables x and y. X containing input variables and y containing target variables for labels

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state = 1234
) # Sets the training set to be 70 percent and 30 oercent will be used as the test set. Random state sets the random number generator to a number specified 

clf = DecisionTree()
clf.fit(x_train, y_train) # Trains the decision tree using the training data
predictions = clf.predict(x_test) # Used to classify the data 

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test) # Produces the accuracy of the algorithm(.10 means 10 percent and so on) 
    

acc = accuracy(y_test, predictions)
print(acc)

0.9590643274853801


In [52]:
class RandomForest:
    def __init__(self, n_trees = 10, max_depth = 10, min_samples_split = 2, n_features = None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features

    def fit(self, x, y):
        self.trees = []
        for _ in range(self.n_trees): # Creates and trains individual decision trees
            tree = DecisionTree(max_depth = self.max_depth, 
                         min_samples_split = self.min_samples_split, 
                         n_features = self.n_features) # A new instance is created for each interation of the loop
            
            x_sample, y_sample = self._bootstrap_samples(x, y)
            tree.fit(x_sample, y_sample) # This trains the decision tree on the sampled data
            self.trees.append(tree) # Appends to the list self.trees after the decision tree is trained

    def _bootstrap_samples(self, x, y):
        n_samples = x.shape[0] # Gets the number of samples in thne dataset
        idxs = np.random.choice(n_samples, n_samples, replace=True) # Generates a random sample of indices with replacement
        return x[idxs], y[idxs]
    
    def predict(self, x):
        predictions = np.array([tree.predict(x) for tree in self.trees]) # Uses a list complrehension to  call the predict method on each decision tree in the random forest
        tree_preds = np.swapaxes(predictions, 0, 1) # Used to swap the axes pf the predictions array
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds]) # Determines the most common prediction for each sample across all the trees
        return predictions 
    
    def _most_common_label(self, y):
        counter = Counter(y) 
        most_common = counter.most_common(1)[0][0] 
        return most_common
    


In [53]:
data = datasets.load_breast_cancer()
x, y = data.data, data.target

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state = 2345
)

clf = RandomForest()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)
    

acc = accuracy(y_test, predictions)
print(acc)

0.9532163742690059
